# Connectivity Mapping using Expert-derived Resistance Rasters

## Rasterise Focal Nodes

In [1]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
import os

# Define paths for focal node shapefiles
focal_node_paths = {
    "Bufo": "C:/GIS_Course/MScThesis-MaviSantarelli/data/FocalNodes/Bufo_Spawning.shp",
    "Rana": "C:/GIS_Course/MScThesis-MaviSantarelli/data/FocalNodes/Rana_Spawning.shp",
    "Lissotriton": "C:/GIS_Course/MScThesis-MaviSantarelli/data/FocalNodes/Liss_Spawning.shp"
}

# Define output ASC paths
output_asc_paths = {
    "Bufo": "C:/GIS_Course/MScThesis-MaviSantarelli/results/FocalNodes/Focal_Nodes_Bufo.asc",
    "Rana": "C:/GIS_Course/MScThesis-MaviSantarelli/results/FocalNodes/Focal_Nodes_Rana.asc",
    "Lissotriton": "C:/GIS_Course/MScThesis-MaviSantarelli/results/FocalNodes/Focal_Nodes_Lissotriton.asc"
}

# Make sure the output directory exists
os.makedirs("C:/GIS_Course/MScThesis-MaviSantarelli/results/FocalNodes", exist_ok=True)

# Reference resistance raster (for alignment)
resistance_raster_path = "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Exponential_Bufo.tif"


In [2]:
# Open resistance raster to get metadata
with rasterio.open(resistance_raster_path) as src:
    meta = src.meta.copy()  # Copy metadata
    raster_shape = src.shape  # Get raster dimensions (rows, cols)
    raster_transform = src.transform  # Get raster extent and resolution


In [3]:
# Loop through each species' focal node shapefile
for species, focal_path in focal_node_paths.items():
    
    # Load focal nodes
    focal_nodes = gpd.read_file(focal_path)
    
    # Rasterizing focal nodes (1 = spawning site, 0 = background)
    focal_nodes_raster = rasterize(
        [(geom, 1) for geom in focal_nodes.geometry],  # Assign value 1 to spawning points
        out_shape=raster_shape,  # Match resistance raster size
        transform=raster_transform,  # Align with raster extent
        fill=0,  # Background = 0 (non-focal areas)
        all_touched=True,  # If points fall between pixels, include them
        dtype=rasterio.uint8  # 8-bit integer for Circuitscape
    )

    print(f"✅ Rasterized focal nodes for {species}.")


✅ Rasterized focal nodes for Bufo.
✅ Rasterized focal nodes for Rana.
✅ Rasterized focal nodes for Lissotriton.


In [4]:
# Loop through each species and save as ASCII Grid
for species, asc_path in output_asc_paths.items():
    asc_meta = meta.copy()
    asc_meta.update(driver="AAIGrid", dtype=rasterio.uint8, nodata=0)  # NoData set to 0

    with rasterio.open(asc_path, "w", **asc_meta) as dst:
        dst.write(focal_nodes_raster.astype(rasterio.uint8), 1)

    print(f"✅ {species} focal nodes raster saved at: {asc_path}")


✅ Bufo focal nodes raster saved at: C:/GIS_Course/MScThesis-MaviSantarelli/results/FocalNodes/Focal_Nodes_Bufo.asc
✅ Rana focal nodes raster saved at: C:/GIS_Course/MScThesis-MaviSantarelli/results/FocalNodes/Focal_Nodes_Rana.asc
✅ Lissotriton focal nodes raster saved at: C:/GIS_Course/MScThesis-MaviSantarelli/results/FocalNodes/Focal_Nodes_Lissotriton.asc


## Create Configuration Files

In [6]:
import os

# Define base directory for Circuitscape configurations
base_dir = "C:/GIS_Course/MScThesis-MaviSantarelli/results/Circuitscape_Configs"
os.makedirs(base_dir, exist_ok=True)

# Define species and resistance models
species_list = ["Bufo", "Rana", "Lissotriton"]
resistance_types = ["Log", "Inv", "Linear", "Exponential", "Categorical"]

# Define resistance raster paths (adjust paths if needed)
resistance_raster_paths = {
    "Bufo": {
        "Log": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Log_Bufo.tif",
        "Inv": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Inv_Bufo.tif",
        "Linear": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Linear_Bufo.tif",
        "Exponential": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Exponential_Bufo.tif",
        "Categorical": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Categorical_Bufo.tif"
    },
    "Rana": {
        "Log": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Log_Rana.tif",
        "Inv": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Inv_Rana.tif",
        "Linear": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Linear_Rana.tif",
        "Exponential": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Exponential_Rana.tif",
        "Categorical": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Categorical_Rana.tif"
    },
    "Lissotriton": {
        "Log": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Log_Lissotriton.tif",
        "Inv": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Inv_Lissotriton.tif",
        "Linear": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Linear_Lissotriton.tif",
        "Exponential": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Exponential_Lissotriton.tif",
        "Categorical": "C:/GIS_Course/MScThesis-MaviSantarelli/results/Resistance/Resistance_Categorical_Lissotriton.tif"
    }
}

# Define focal node raster paths
focal_node_paths = {
    "Bufo": "C:/GIS_Course/MScThesis-MaviSantarelli/results/FocalNodes/Focal_Nodes_Bufo.asc",
    "Rana": "C:/GIS_Course/MScThesis-MaviSantarelli/results/FocalNodes/Focal_Nodes_Rana.asc",
    "Lissotriton": "C:/GIS_Course/MScThesis-MaviSantarelli/results/FocalNodes/Focal_Nodes_Lissotriton.asc"
}


In [7]:
# Create Circuitscape `.ini` file for each species & resistance type
for species in species_list:
    for resistance in resistance_types:
        # Define output `.ini` file path
        ini_path = os.path.join(base_dir, f"{species}_{resistance}.ini")
        
        # Optimized Circuitscape Configuration
        config = f"""[Circuitscape]
data_type = raster
scenario = pairwise

[Habitat raster or graph]
habitat_file = {resistance_raster_paths[species][resistance]}
habitat_map_is_resistances = True

[Options for pairwise and one-to-all and all-to-one modes]
point_file = {focal_node_paths[species]}

[Output options]
output_file = {base_dir}/{species}_{resistance}_output
write_cur_maps = True
write_volt_maps = False
write_cum_cur_map_only = True
write_max_cur_maps = False
log_transform_maps = False
set_null_currents_to_nodata = True
set_null_voltages_to_nodata = True
compress_grids = True

[Calculation options]
low_memory_mode = True
solver = amg+cg  # Efficient for large rasters
print_timings = False

[Connection scheme for raster habitat data]
connect_four_neighbors_only = False  # Allow diagonal movement
connect_using_avg_resistances = False

[Options for one-to-all and all-to-one modes]
use_variable_source_strengths = False
variable_source_file = None
"""
        # Save `.ini` file
        with open(ini_path, "w") as file:
            file.write(config)
        
        print(f"✅ Optimized Circuitscape config saved: {ini_path}")


✅ Optimized Circuitscape config saved: C:/GIS_Course/MScThesis-MaviSantarelli/results/Circuitscape_Configs\Bufo_Log.ini
✅ Optimized Circuitscape config saved: C:/GIS_Course/MScThesis-MaviSantarelli/results/Circuitscape_Configs\Bufo_Inv.ini
✅ Optimized Circuitscape config saved: C:/GIS_Course/MScThesis-MaviSantarelli/results/Circuitscape_Configs\Bufo_Linear.ini
✅ Optimized Circuitscape config saved: C:/GIS_Course/MScThesis-MaviSantarelli/results/Circuitscape_Configs\Bufo_Exponential.ini
✅ Optimized Circuitscape config saved: C:/GIS_Course/MScThesis-MaviSantarelli/results/Circuitscape_Configs\Bufo_Categorical.ini
✅ Optimized Circuitscape config saved: C:/GIS_Course/MScThesis-MaviSantarelli/results/Circuitscape_Configs\Rana_Log.ini
✅ Optimized Circuitscape config saved: C:/GIS_Course/MScThesis-MaviSantarelli/results/Circuitscape_Configs\Rana_Inv.ini
✅ Optimized Circuitscape config saved: C:/GIS_Course/MScThesis-MaviSantarelli/results/Circuitscape_Configs\Rana_Linear.ini
✅ Optimized Circui